In [74]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [107]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer

In [14]:
df = pd.read_csv('default of credit card clients.xls - Data.csv')

In [15]:
df.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0


In [17]:
df.index

RangeIndex(start=0, stop=30001, step=1)

In [18]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.drop([0], axis=0, inplace=True)

In [19]:
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [20]:
y = df.Y
X = df.drop(['Y'], axis=1)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5)

/anaconda3/envs/DataScience36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2069: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [22]:
num_cols = ['X1', 'X5', 'X12', 'X13','X14','X15','X16','X17', 'X18','X19','X20','X21','X22','X23',]
cat_cols = ['X2', 'X3', 'X4', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11']

In [23]:
# Preprocessing pipeline
preprocessing_nums = Pipeline([
    ('scaler',  StandardScaler())
])
preprocessing_cats = Pipeline([
    ('dummy',   OneHotEncoder(handle_unknown='ignore')), 
])

preprocessing = ColumnTransformer([
    ('nums',   preprocessing_nums,  num_cols),
    ('cats',   preprocessing_cats,  cat_cols)
],
    remainder='drop', n_jobs=-1)

In [25]:
pipe = Pipeline([
    ('preprocessing', preprocessing),
    ('model', LogisticRegression())], 
    memory='cachedir') 

In [34]:
params = [
    { 'model': [LogisticRegression()],
      'model__C': [.1, 1]
    },
    { 'model': [GaussianNB()]
    },
    { 'model': [KNeighborsClassifier()],
      'model__n_neighbors': [1,3,5,7]
    },
]

In [35]:
grid = GridSearchCV(estimator=pipe, param_grid=params, scoring='roc_auc',
                          cv=3, n_jobs=-1, verbose=4, error_score=np.nan,
                          return_train_score=False)

In [36]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done  19 out of  27 | elapsed:   14.0s remaining:    5.9s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:   18.8s finished
/anaconda3/envs/DataScience36/lib/python3.6/site-packages/sklearn/pipeline.py:230: UserWarning: Persisting input arguments took 1.13s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])
/anaconda3/envs/DataScience36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

GridSearchCV(cv=3, error_score=nan,
       estimator=Pipeline(memory='cachedir',
     steps=[('preprocessing', ColumnTransformer(n_jobs=-1, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('nums', Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True))]), ['X1', 'X5', 'X12', 'X13', 'X14',...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'model': [LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)], 'model__C': [..._jobs=None, n_neighbors=5, p=2,
           weights='uniform')], 'model__n_neighbors': [1, 3, 5, 7]}],
       pre_dispatch='2*n_jobs', refit=True, retur

In [37]:
# summarize results
print("Best: {:.4f} using {}".format(grid.best_score_, 
    grid.best_params_))
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']

Best: 0.7619 using {'model': LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False), 'model__C': 1}


In [40]:
results = pd.DataFrame([means, stds, params], index=['Media', 'Stdev', 'Params']).T
results = results.sort_values(by=['Media', 'Stdev'], ascending=[False, True])
results['Media']=results['Media'].map('{:,.4f}'.format)
results['Stdev']=results['Stdev'].map('{:,.4f}'.format)
results

,Media,Stdev,Params
0,0.7619,0.0011,"{'model': LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', random_state=None, solver='warn', tol=0.0001, verbose=0, warm_start=False), 'model__C': 1}"
1,0.7617,0.0012,"{'model': LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', random_state=None, solver='warn', tol=0.0001, verbose=0, warm_start=False), 'model__C': 5}"
2,0.7616,0.0012,"{'model': LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', random_state=None, solver='warn', tol=0.0001, verbose=0, warm_start=False), 'model__C': 10}"
3,0.7612,0.0011,"{'model': LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=None, penalty='l2', random_state=None, solver='warn', tol=0.0001, verbose=0, warm_start=False), 'model__C': 100}"
8,0.6964,0.0061,"{'model': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski', metric_params=None, n_jobs=None, n_neighbors=5, p=2, weights='uniform'), 'model__n_neighbors': 7}"
7,0.6829,0.0069,"{'model': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski', metric_params=None, n_jobs=None, n_neighbors=5, p=2, weights='uniform'), 'model__n_neighbors': 5}"
6,0.6531,0.0091,"{'model': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski', metric_params=None, n_jobs=None, n_neighbors=5, p=2, weights='uniform'), 'model__n_neighbors': 3}"
5,0.6012,0.0084,"{'model': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski', metric_params=None, n_jobs=None, n_neighbors=5, p=2, weights='uniform'), 'model__n_neighbors': 1}"
4,nan,nan,"{'model': GaussianNB(priors=None, var_smoothing=1e-09)}"


In [41]:
log_pipe = Pipeline(
[('prepro', preprocessing),
 ('model',  LogisticRegression(C=.1))]
)

In [43]:
log_pipe.fit(X_train, y_train)

/anaconda3/envs/DataScience36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('prepro', ColumnTransformer(n_jobs=-1, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('nums', Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True))]), ['X1', 'X5', 'X12', 'X13', 'X14', 'X15',...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [73]:
p_pred = log_pipe.predict_proba(X_test)[:, 1]
y_pred = log_pipe.predict(X_test)

In [75]:
p_pred
p_pred.shape
type(p_pred)

array([0.15331351, 0.11046038, 0.4460503 , ..., 0.14954436, 0.0928534 ,
       0.11948162])

(15000,)

numpy.ndarray

In [77]:
y_test.values
y_test.shape
type(y_test)

array(['0', '0', '0', ..., '0', '0', '0'], dtype=object)

(15000,)

pandas.core.series.Series

In [78]:
p_pred_df = pd.DataFrame(p_pred)
y_test_df = pd.DataFrame(y_test.values)

In [81]:
res = pd.concat([y_test_df, p_pred_df], axis=1, ignore_index=True)
res.columns = ['y_true', 'prob']
res.head(20)

,y_true,prob
0,0,0.153314
1,0,0.110460
2,0,0.446050
3,0,0.050465
4,0,0.165569
5,1,0.151920
6,0,0.017664
7,0,0.145180
8,0,0.093725
9,1,0.157824


In [82]:
res_sorted = res.sort_values(by='prob', ascending=True)

In [83]:
res_sorted.head()
res_sorted.tail()

,y_true,prob
1764,0,2.789457e-15
4736,0,3.415376e-11
10146,0,9.914978e-06
14548,0,2.194810e-04
3662,0,3.269586e-04


,y_true,prob
7103,0,0.888163
13388,0,0.890254
8335,1,0.894460
7826,1,0.895716
4508,0,0.895795


In [85]:
# metrics
roc_auc_score(y_test, p_pred)

0.7735952824605429

In [92]:
res.reset_index(inplace=True)

In [103]:
def SSM(sorted_y, n=5):
    out=[]
    l=len(sorted_y)
    for i in range(l):
        out.append(0)
        ni=0
        for j in range(i-n, i+n+1):
            if j>=0 and j<l:
                out[i] += sorted_y[j]
                ni+=1
        out[i]/=ni
    return out

In [104]:
res['y_true'] = res['y_true'].astype('int')

In [106]:
p_prob_real = SSM(res.y_true.values)

In [110]:
lin_mod = LinearRegression()
lin_mod.fit(res.prob.values.reshape(-1, 1), p_prob_real)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [113]:
lin_mod.intercept_, lin_mod.coef_

(0.20113732319299962, array([0.0879534]))

In [117]:
lin_mod.score(res.prob.values.reshape(-1, 1), p_prob_real)

0.018016974039351315

In [ ]:
sns.lmplot(x="total_bill", y="tip", data=tips);